# Importing the files to mysql

In [1]:
import pandas as pd
import os

import matplotlib.pyplot as plt 
import seaborn as sns 
from sqlalchemy import create_engine
from urllib.parse import quote_plus

import plotly.express as px 
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors 
pio.templates.default = 'plotly_white'

In [2]:
username = 'root'
password = quote_plus('Yashasvi@1234')
host = 'localhost'
port = 3306
database = 'full_project'


In [3]:
engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
)


In [4]:
csv_folder = r"D:\Full DA Project"

In [5]:
for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(csv_folder, file)
        table_name = os.path.splitext(file)[0]

        df = pd.read_csv(file_path)

        # Clean column names
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(" ", "_")
            .str.replace("-", "_")
        )

        # NaN → NULL (THIS must be here)
        df = df.where(pd.notnull(df), None)

        df.to_sql(
            name=table_name,
            con=engine,
            if_exists="replace",
            index=False
        )

        print(f"Imported: {file} → table `{table_name}`")


Imported: customers.csv → table `customers`
Imported: geolocation.csv → table `geolocation`
Imported: orders.csv → table `orders`
Imported: order_items.csv → table `order_items`
Imported: payments.csv → table `payments`
Imported: products.csv → table `products`
Imported: sellers.csv → table `sellers`


In [6]:
def get_sqlalchemy_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return Integer()
    elif pd.api.types.is_float_dtype(dtype):
        return Float()
    elif pd.api.types.is_bool_dtype(dtype):
        return Boolean()
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return DateTime()
    else:
        return Text()


# BASIC

## 1. List all unique cities where customers are located.


In [7]:
query = """ select distinct customer_city from customers"""
df = pd.read_sql(query , engine )
df.head()

,customer_city
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas


## 2. Count the number of orders placed in 2017.

In [8]:
query  = """ select year(order_purchase_timestamp) as year , count(order_id) as count from orders 
group by year
having year = 2017
 """

df = pd.read_sql(query , engine )
df 

,year,count
0,2017,45101


## 3. Find the total sales per category.

In [9]:
query = """ select p.product_category , 
sum(pay.payment_value) as total_sales
from order_items as oi 
join products as p 
on oi.product_id = p.product_id 
join payments as pay 
on pay.order_id = oi.order_id
group by p.product_category
order by sum(pay.payment_value) desc ;

"""

df = pd.read_sql (query , engine )
df

,product_category,total_sales
0,bed table bath,1712553.67
1,HEALTH BEAUTY,1657373.12
2,computer accessories,1585330.45
3,Furniture Decoration,1430176.39
4,Watches present,1429216.68
...,...,...
69,PC Gamer,2174.43
70,House Comfort 2,1710.54
71,cds music dvds,1199.43
72,Fashion Children's Clothing,785.67


In [10]:
fig = px.bar(df , x = 'product_category' , y = 'total_sales' ,
             title= 'Total Sales by Category')
fig.show()

## 4. Calculate the percentage of orders that were paid in installments.

In [11]:
query = """ select 
(sum(case when payment_installments > 1 then 1 else 0  end )/ count(payment_installments))*100
as 'installment payments percentage '
from payments;
"""


df1 = pd.read_sql(query , engine)
df1

,installment payments percentage
0,49.4176


## 5. Count the number of customers from each state.

In [12]:
query = """ select  customer_state  ,count(distinct customer_id )  as no_of_customers
from customers 
group by customer_state """

df = pd.read_sql(query , engine)
df.set_index('customer_state' , inplace = True)
df 

,no_of_customers
customer_state,
AC,81
AL,413
AM,148
AP,68
BA,3380
CE,1336
DF,2140
ES,2033
GO,2020


In [13]:
fig = px.pie( df , df.index , values='no_of_customers',
             hole= 0.4 , title = "CUSTOMER'S STATE")
fig.update_traces(textposition  =  'inside' , textinfo='label+percent')
fig.update_layout(width= 500, height = 500,
    showlegend = False)
fig.show()

# INTERMEDIATE

## 1. Calculate the number of orders per month in 2018.

In [47]:
query = """ select month(order_purchase_timestamp) as month
, count(order_id) as 'number of orders' from orders
where year(order_purchase_timestamp) = 2018
group by month(order_purchase_timestamp)
order by month(order_purchase_timestamp)
"""

df = pd.read_sql(query , engine)
df.set_index ('month' , inplace= True)
df 

,number of orders
month,
1,7269
2,6728
3,7211
4,6939
5,6873
6,6167
7,6292
8,6512
9,16


In [57]:
fig = px.line(
    df,
    x=df.index,
    y='number of orders'
)

fig.update_traces(line_color='#8E44AD')

fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        dtick=1  
    ),
    width = 700, 
    height = 250
)

fig.show()


## 2. Find the average number of products per order, grouped by customer city.

In [16]:
query = """select customer_city , round(avg(xyz),2)  as average_orders from
( select c.customer_city , count(*) as xyz
from customers as c 
join orders as o 
on c.customer_id = o.customer_id 
join order_items as oi 
on oi.order_id = o.order_id
group by c.customer_city , oi.order_id )as t 
group by t.customer_city 
order by average_orders desc;"""

df = pd.read_sql(query, engine)
df

,customer_city,average_orders
0,padre carvalho,7.0
1,celso ramos,6.5
2,datas,6.0
3,candido godoi,6.0
4,matias olimpio,5.0
...,...,...
4105,garruchos,1.0
4106,humberto de campos,1.0
4107,bela vista do maranhao,1.0
4108,cantanhede,1.0


## 3. Calculate the percentage of total revenue contributed by each product category.

In [17]:
query = """ select prod.product_category ,round((sum(pay.payment_value)*100/
(select sum(payment_value) from payments)),2) as revenue
from products as prod 
join order_items as oi
on prod.product_id = oi.product_id
join payments as pay 
on pay.order_id = oi.order_id 
group by prod.product_category  
order by revenue desc """

df = pd.read_sql(query , engine)
df 

,product_category,revenue
0,bed table bath,10.70
1,HEALTH BEAUTY,10.35
2,computer accessories,9.90
3,Furniture Decoration,8.93
4,Watches present,8.93
...,...,...
69,House Comfort 2,0.01
70,cds music dvds,0.01
71,PC Gamer,0.01
72,Fashion Children's Clothing,0.00


In [18]:
top = df.head(30 )
top

,product_category,revenue
0,bed table bath,10.70
1,HEALTH BEAUTY,10.35
2,computer accessories,9.90
3,Furniture Decoration,8.93
4,Watches present,8.93
5,sport leisure,8.70
6,housewares,6.84
7,automotive,5.32
8,Garden tools,5.24
9,Cool Stuff,4.87


In [19]:
fig = px.bar(top , x = 'product_category' ,y = 'revenue',color = 'revenue', 
             color_continuous_scale= 'Inferno_r',
             text = 'revenue', title= 'Revenue by Category')
fig.update_traces( textposition = 'inside')
fig.show()


## 4. Identify the correlation between product price and the number of times a product has been purchased.

In [20]:
query = """ select p.product_category , count(o.product_id) as 
number_of_purchases , avg(o.price) as price
from order_items as o 
join products as  p
on p.product_id = o.product_id
group by p.product_category 
order by number_of_purchases desc
"""

df = pd.read_sql(query, engine)
df


,product_category,number_of_purchases,price
0,bed table bath,11115,93.296327
1,HEALTH BEAUTY,9670,130.163531
2,sport leisure,8641,114.344285
3,Furniture Decoration,8334,87.564494
4,computer accessories,7827,116.513903
...,...,...,...
69,La Cuisine,14,146.785000
70,cds music dvds,14,52.142857
71,PC Gamer,9,171.772222
72,Fashion Children's Clothing,8,71.231250


## 5. Calculate the total revenue generated by each seller, and rank them by revenue.


In [21]:
query = """ select * , rank() over(order by revenue desc) as rn from 
(select s.seller_id , sum(o.price) as revenue
from sellers as s 
join order_items as o 
on s.seller_id = o.seller_id
group by s.seller_id
order by revenue desc) as t """

df = pd.read_sql(query , engine)
df

,seller_id,revenue,rn
0,4869f7a5dfa277a7dca6462dcf3b52b2,229472.63,1
1,53243585a1d6dc2643021fd1853d8905,222776.05,2
2,4a3ca9315b744ce9f8e9374361493884,200472.92,3
3,fa1c13f2614d7b5c4749cbc52fecda94,194042.03,4
4,7c67e1448b00f6e969d365cea6b010ab,187923.89,5
...,...,...,...
3090,34aefe746cd81b7f3b23253ea28bef39,8.00,3091
3091,702835e4b785b67a084280efca355756,7.60,3092
3092,1fa2d3def6adfa70e58c276bb64fe5bb,6.90,3093
3093,77128dec4bec4878c37ab7d6169d6f26,6.50,3094


In [22]:
fig = px.bar(df.head(10), x = 'seller_id', y = 'revenue', title= 'Top 10 sellers' , 
             color= 'revenue',  color_continuous_scale= 'Viridis_r'
             )
fig.show()

# ADVANCE

## 1. Calculate the moving average of order values for each customer over their order history.

In [23]:
query = """ 
select customer_id, order_purchase_timestamp, payment,
avg(payment) over(partition by customer_id order by order_purchase_timestamp
rows between 2 preceding and current row) as mov_avg
from
(select orders.customer_id, orders.order_purchase_timestamp, 
payments.payment_value as payment
from payments join orders
on payments.order_id = orders.order_id) as a
 ; """

df = pd.read_sql(query ,engine)
df

,customer_id,order_purchase_timestamp,payment,mov_avg
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.74
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,67.41,67.41
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,195.42,195.42
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,179.35,179.35
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,107.01,107.01
...,...,...,...,...
103881,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26,9.49,27.12
103882,fffeda5b6d849fbd39689bb92087f431,2018-05-22 13:36:02,63.13,63.13
103883,ffff42319e9b2d713724ae527742af25,2018-06-13 16:57:05,214.13,214.13
103884,ffffa3172527f765de70084a7e53aae8,2017-09-02 11:53:32,45.50,45.50


## 2. Calculate the cumulative sales per month for each year.

In [24]:
query = """ select * , sum(payments) over( order by years , months) as cum_sum from
(select  year(o.order_purchase_timestamp) as years ,
 month(o.order_purchase_timestamp)  as months ,
 round(sum(p.payment_value),2) as payments from 
orders as o 
join payments as p
on o.order_id = p.order_id 
group by  years ,months
order by years , months ) as t
 ;"""

df = pd.read_sql(query , engine)
df 

,years,months,payments,cum_sum
0,2016,9,252.24,252.24
1,2016,10,59090.48,59342.72
2,2016,12,19.62,59362.34
3,2017,1,138488.04,197850.38
4,2017,2,291908.01,489758.39
5,2017,3,449863.60,939621.99
6,2017,4,417788.03,1357410.02
7,2017,5,592918.82,1950328.84
8,2017,6,511276.38,2461605.22
9,2017,7,592382.92,3053988.14


## 3. Calculate the year-over-year growth rate of total sales.

In [25]:
query = """ with a as (with t as (select year(o.order_purchase_timestamp) 
as years , round(sum(p.payment_value),2) as sales 
from orders as o 
join payments as p 
on o.order_id = p.order_id
group by years 
order by years)

select * , lag(sales ,1 ) over(order by years) as prev_sales 
from t ) 

select years , round(((sales-prev_sales)*100/prev_sales ),2) as yoy_growth 
from a
;"""

df = pd.read_sql(query , engine)
df 

,years,yoy_growth
0,2016,NaN
1,2017,12112.7
2,2018,20.0


## 4. Calculate the cumulative sales over yers 

In [26]:
query = """select * , round(sum(sales) over(order by years),2) as 
cum_sum from
(select year(o.order_purchase_timestamp) 
as years , round(sum(p.payment_value),2) as sales 
from orders as o 
join payments as p 
on o.order_id = p.order_id
group by years 
order by years) as t"""

df = pd.read_sql(query ,engine )
df

,years,sales,cum_sum
0,2016,59362.34,59362.34
1,2017,7249746.73,7309109.07
2,2018,8699763.05,16008872.12


## 5. Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase.

In [27]:
query = """ with b as (with a as (select customer_id , min(order_purchase_timestamp) as first_order 
from orders 
group by customer_id) 
select  a.customer_id , count(o.order_purchase_timestamp) as frequency
from a 
join orders as o 
on o.customer_id = a.customer_id 
where o.order_purchase_timestamp > a.first_order 
and o.order_purchase_timestamp < date_add(a.first_order , interval 6 month )
group by a.customer_id )

select count(frequency )*100 / count( distinct o.customer_id )
as retentivity 
from b 
join orders as o 
on b.customer_id = o.customer_id 
;"""

df = pd.read_sql(query ,engine)
df

,retentivity
0,None


## 6. Identify the top 3 customers who spent the most money in each year.

In [59]:
query = """ select * from 
(with a as  
(select c.customer_id , sum(p.payment_value) as amount ,
year (o.order_purchase_timestamp) as  years
from customers as c 
join orders as o 
on c.customer_id = o.customer_id 
join payments as p 
on p.order_id = o.order_id 
group by years ,c.customer_id 
order by years  ,amount desc ) 


select * , rank() over(partition by years  order by amount desc) as rn 
from a ) as b 

where rn <= 3
;

"""

df = pd.read_sql(query , engine )
df 

,customer_id,amount,years,rn
0,a9dc96b027d1252bbac0a9b72d837fc6,1423.55,2016,1
1,1d34ed25963d5aae4cf3d7f3a4cda173,1400.74,2016,2
2,4a06381959b6670756de02e07b83815f,1227.78,2016,3
3,1617b1357756262bfa56ab541c47bc16,13664.08,2017,1
4,c6e2731c5b391845f6800c97401a43a9,6929.31,2017,2
5,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66,2017,3
6,ec5b2ba62e574342386871631fafd3fc,7274.88,2018,1
7,f48d464a0baaea338cb25f816991ab1f,6922.21,2018,2
8,e0a2412720e9ea4f26c1ac985f6a7358,4809.44,2018,3


In [63]:
df['years'] = df['years'].astype(str)
# plotly shows gradient on numeric value in  legends so we need to convert it to string type  

fig = px.bar(df , x = 'customer_id' , y = 'amount' ,text = 'amount',
              color= 'years' ,
              color_discrete_sequence= px.colors.qualitative.Dark24)

fig.update_layout(
    width= 1100,
    height = 600,
    title ={ 'text': '<b> Top 3 Customers each year <b>',
            'x': 0.5,
             'xanchor': 'center',
              'font': {'size':20,
                       'family': 'arial',
                        'color':'black' }

                       }

)
fig.show()